# dataset + sqlite3

### context management for transactions in data enviroments


**GOAL**

> * Being able to keep track of a downloads, paths (location), and the name-of-files without having to explicitly import, load, and save back-to-back repeating the same steps multiple times.

*When calling e.g. the `download(video_id)` method, which can return a: `(path, file-name, and meta-info)` where the file-name is the : **`video_id`**, meta-info : **'comment_count'** (*count is available and int-value can be returned*).*


> * The database should hold the folowing values (all the following values can be obtained from the *`david.youtube.scraper.download`* method):
    
- file_path : str

- file_name : str
        
- file_meta : int

In [34]:
import sys
sys.path.append('/home/ego/Github/david/')

import os
from os.path import exists, join, isfile
from collections import namedtuple

import dataset
import pandas as pd

from david.utils import pointer
from david.youtube import scraper

#### Connecting to a database

> It is also possible to define the URL as an environment variable called `DATABASE_URL` so you can initialize database connection without explicitly passing an URL:

```python
# connecting to a SQLite database
db = dataset.connect('sqlite:///mydatabase.db')
```

#### Storing data

> To store some data you need to get a reference to a table. You don’t need to worry about whether the table already exists or not, since dataset will create it automatically:

* Insert a new record.

```python
# get a reference to the table 'user'.
table = db['user']
table.insert(dict(name='John Doe', age=46, country='China'))

# dataset will create "missing" columns any time you insert a dict with an unknown key.
table.insert(dict(name='Jane Doe', age=37, country='France', gender='female'))
```

* Updating existing entries.

> The list of filter columns given as the second argument filter using the values in the first column. If you don’t want to update over a particular value, just use the auto-generated id column.

```python
table.update(dict(name='John Doe', age=47), ['name'])
```

In [2]:
DATABASE_URL = 'sqlite:///context_manager.db'
CONTEXT_TABLE = 'context'

db = dataset.connect(DATABASE_URL)

# when a transaction is executed: e.g user calling scraper.downlod(some-video, count=100)
# the following parameters are created and all three parameters are ALWAYS expected.

trans_1 = dict(path='downloads', name='4Dk3jOSbz_0', entries=100) # meta = entries
trans_2 = dict(path='downloads', name='BmYZH7xt8sU', entries=4252)
trans_1, trans_2

({'path': 'downloads', 'name': '4Dk3jOSbz_0', 'entries': 100},
 {'path': 'downloads', 'name': 'BmYZH7xt8sU', 'entries': 4252})

In [3]:
# pass the keyword arguments to the database and save
# (i need to add a time of download index!)

table = db[CONTEXT_TABLE]
table.insert(trans_1)
table.insert(trans_2)
db.commit()
table.columns

['id', 'path', 'name', 'entries']

In [4]:
# creates the datable if it doest exists at the address
%ls *.db

context_manager.db


#### Using Transactions

> You can group a set of database updates in a transaction. In that case, all updates are committed at once or, in case of exception, all of them are reverted. Transactions are supported through a context manager, so they can be used through a with statement:

#### Storing data

> To store some data you need to get a reference to a table. You don’t need to worry about whether the table already exists or not, since dataset will create it automatically:

* Insert a new record.

```python
with dataset.connect() as tx:
    tx['user'].insert(dict(name='John Doe', age=46, country='China'))

# you can get same functionality by invoking the methods:
# begin(), commit() and rollback() explicitly:

db = dataset.connect()
db.begin()
try:
    db['user'].insert(dict(name='John Doe', age=46, country='China'))
    db.commit()
except:
    db.rollback()

# nested transactions are supported too:
db = dataset.connect()
with db as tx1:
    tx1['user'].insert(dict(name='John Doe', age=46, country='China'))
    with db as tx2:
        tx2['user'].insert(dict(name='Jane Doe', age=37, country='France', gender='female'))
```

In [5]:
from collections import namedtuple
from collections import Counter

def context_pointer(name: str, *args):
    '''
    * Creating a new pointer: returns an instance class like object.

        >>> File = context_pointer('File', ['path', 'name', 'entries'])
        >>> File.__doc__
         'File(path, name, entries)'

        >>> file = File(path='downloads', name='vdsjhdsj11', entries=30)
         File(path='downloads', name='vdsjhdsj11', entries=30)
    
    * By calling the `_asdict()` method returns a dict object.

        >>> file_dict = file._asdict()
        >>> file_dict['name']
         'vdsjhdsj11'
    '''
    return namedtuple(name, *args)

File = context_pointer('File', ['path', 'name', 'entries'])

file = File(path='downloads', name='vdsjhdsj11', entries=30)
file

File(path='downloads', name='vdsjhdsj11', entries=30)

In [21]:
with open('pending_downloads.txt', 'r', encoding='utf-8') as f:
    videos = []
    for line in f.readlines():
        videos.append(line.strip('\n'))
    f.close()

In [23]:
len(videos)

28

In [25]:
# create a new table in the database
pending = db['pending_downloads']

In [28]:
# insert all video ids into with insert many
for vid in videos:
    pending.insert_many([dict(video_id=vid)])

In [33]:
# we can now query the videos from the database.
for vid in db['pending_downloads']:
    print(vid['video_id'])

2FmcHiLCwTU
bhxhNIQBKJI
f9573kGBtuE
XnbCSboujF4
ad_higXixRA
gf_MqDBBMPI
EcojyBV4QJ4
R2pIutTspQA
XOLOLrUBRBY
seJSm-vD9OA
pU9iJB0ATpk
voYuH5eGlkk
6DasnKpMyQ8
u4ZoJKF_VuA
Ye7FKc1JQe4
quouML5-UZI
pOmu0LtcI6Y
gpST3RcbRjg
ees_hypDprw
jBHy3M4g9vA
JRIPV0dPAd4
iVpXrbZ4bnU
HXhbnC2ooE0
dMF2i3A9Lzw
aI6bPspVOmU
0SPwwpruGIA
bKOTKHtbM54
h0oZI-FvrFk


In [37]:
db = dataset.connect('sqlite:///yt_comments.db')
comments_table = db['comments']

In [40]:
from sys import stdout

def download_batch(videoid: str, table='comments', db_url='sqlite:///yt_comments.db'):
    count = 0
    with dataset.connect(db_url) as db:
        for comment in scraper._scrape_comments(videoid):
            db[table].insert(dict(
                cid=comment['cid'], text=comment['text'],
                time=comment['time'], author=comment['author']
            ))
            count += 1
            stdout.write('mining %d comment(s)\r' % count)
            stdout.flush()

In [41]:
download_batch(video_ids=videos)

/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/dataset/table.py:218: RuntimeWarning: Changing the database schema inside a transaction in a multi-threaded environment is likely to lead to race conditions and synchronization issues.
  RuntimeWarning)


Exception during reset or similar
Traceback (most recent call last):
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 500, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140083100628800 and this is thread id 140083022698240.


Exception closing connection <sqlite3.Connection object at 0x7f6768d85730>
Traceback (most recent call last):
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 500, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140083100628800 and this is thread id 140083022698240.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 270, in _close_connection
    self._